In [23]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [25]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='6um'

In [26]:
design.get_chip_z('Q_chip')

'6um'

In [27]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_width='370um',
                                            pad_height='100um',
                                            pad_gap='80um',
                                            pocket_width='680um',
                                            pocket_height='680um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-120}um",
                                              pos_y=f"{q0y+90}um",
                                              pad_width="110um",
                                              pad_height="100um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [28]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [29]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [30]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [31]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length
3.85mm

In [32]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [33]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 +0.5-0.23-0.5# subtract coupler length, for +0.5Ghz, coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.5775622473488387]


In [34]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','700um']
jogs1[3]=['R','200um']
jogs1[4]=['R','670um']
jogs1[5]=['L','200um']
jogs1[6]=['L','740um']


In [35]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '944um', '100um')
            

# EPR Analysis

In [36]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [37]:
hfss.start()

INFO 03:37PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:37PM [load_ansys_project]: 	Opened Ansys App
INFO 03:37PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 03:37PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Flipchip Bump
INFO 03:37PM [connect_design]: 	Opened active design
	Design:    flip7um_V6 [Solution type: Eigenmode]
INFO 03:37PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:37PM [connect]: 	Connected to project "Flipchip Bump" and design "flip7um_V6" 😀 



True

In [38]:
hfss.activate_ansys_design("flip6um_V6", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

03:37PM 52s WARNING [activate_ansys_design]: The design_name=flip6um_V6 was not in active project.  Designs in active project are: 
['flip7um_V6'].  A new design will be added to the project.  
INFO 03:37PM [connect_design]: 	Opened active design
	Design:    flip6um_V6 [Solution type: Eigenmode]
WARNING 03:37PM [connect_setup]: 	No design setup detected.
WARNING 03:37PM [connect_setup]: 	Creating eigenmode default setup.
INFO 03:37PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [39]:
hfss.close()


Warning! 3 COM references still alive
Ansys will likely refuse to shut down


True

# LOM Analysis

In [40]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [43]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.69',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.69',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative',
 'run': {'name': None,
  'components': ['Q0', 'c0'],
  'open_terminations': [('c0', 'in')],
  'box_plus_buffer': True}}

In [45]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 04:24PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpx43tssjp.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5690000000, Maxwell, 1, False
INFO 04:24PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpuc_ffmtq.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5690000000, Maxwell, 1, False
INFO 04:24PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmphgbr1otx.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5690000000, Maxwell, 2, False
INFO 04:24PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpxad_9m1h.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5690000000, Maxwell, 3, False
INFO 04:24PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpxhkn0d5c.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5690000000, Maxwell, 4, False
INFO 04:24

,pad_bot_Q0,pad_top_Q0,quad_coupler_c0,SignalNet1
pad_bot_Q0,139.74000,-11.454,-0.05489,-127.020
pad_top_Q0,-11.45400,139.680,-18.88100,-107.980
quad_coupler_c0,-0.05489,-18.881,96.14800,-77.042
SignalNet1,-127.02000,-107.980,-77.04200,376.670


In [46]:
c1.setup.junctions = Dict({'Lj': 12, 'Cj': 2})
c1.setup.freq_readout = 7.36
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.279409 [GHz]
EC 285.976505 [MHz]
EJ 13.616300 [GHz]
alpha -329.897663 [MHz]
dispersion 17.076274 [KHz]
Lq 11.995161 [nH]
Cq 67.733635 [fF]
T1 286.715256 [us]

**Coupling Properties**

tCqbus1 -4.459920 [fF]
gbus1_in_MHz -68.398078 [MHz]
χ_bus1 -0.635307 [MHz]
1/T1bus1 555.097573 [Hz]
T1bus1 286.715256 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.172656,399.95264,13.6163,-478.124206,308.329885,[-53.693745783290105],[-1.4097413180618719],1.409741,53.693746
2,5.384767,298.282857,13.6163,-345.444257,25.276675,[-58.2615114467913],[-0.5264494583138472],0.526449,58.261511
3,5.336952,292.661077,13.6163,-338.328913,21.196657,[-63.844817219596614],[-0.5949400075401841],0.594940,63.844817
4,5.308856,289.386256,13.6163,-334.194418,19.084947,[-67.13381702201913],[-0.6349724332144817],0.634972,67.133817
5,5.2986,288.196125,13.6163,-332.693752,18.362518,[-67.21684225108443],[-0.628422269250774],0.628422,67.216842
6,5.287991,286.967882,13.6163,-331.146083,17.641105,[-67.3815244547436],[-0.6231925110514621],0.623193,67.381524
7,5.279409,285.976505,13.6163,-329.897663,17.076274,[-68.39807842299159],[-0.6353071622394522],0.635307,68.398078


In [47]:
c1.sim.close()

Warning! 9 COM references still alive
Ansys will likely refuse to shut down
